In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import time
import sys

import wget

from astroquery.mast import Observations

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table

In [2]:
manifest = pd.read_csv('C:/Users/oryan/Documents/esac-project/manifests/gz-hubble-local-manifest.csv',index_col=0)
test_id = 'AHZ20000ak'
row = manifest.query('zooniverse_id == @test_id')
ra = row.RA.iloc[0]
dec = row.Dec.iloc[0]

In [ ]:
folder = f'C:/Users/oryan/Documents/zoobot_new/preprocessed-cutouts/fromMAST/{test_id}/mastDownload/HLA'
folders = glob.glob(folder + '/*')
paths = []
for i in folders:
    files = glob.glob(i + '/*.fits')[0]
    paths.append(files)

In [4]:
file = os.path.basename(files).replace('_drz.fits','')

In [5]:
file = 'hst_10092_5n_acs_wfc_f814w'

In [7]:
url = use_fitscut(file,ra,dec,100,300)

## Doing this for all..

In [3]:
def use_fitscut(file,ra,dec,cutout_size,output_size):
    url = f'https://hla.stsci.edu/cgi-bin/fitscut.cgi?red={file}&RA={ra}&Dec={dec}&size={cutout_size}&output_size={output_size}'
    return url

In [4]:
for i in range(len(manifest)):
    row = manifest.iloc[i]
    ra = row.RA
    dec = row.Dec
    zooniverse_id = row.zooniverse_id
    
    coord = SkyCoord(ra = ra, dec = dec, unit='deg', frame = 'fk5')

    obs_table = Observations.query_criteria(
        coordinates = coord,
        radius = 20 * u.arcsec,
        dataproduct_type='image',
        obs_collection = 'HST',
        calib_level = 3,
        filters = ['F814W']
        )
    
    if len(obs_table) < 0.5:
        continue

    par_obs_id = obs_table[0]

    data_products = Observations.get_product_list(par_obs_id)

    download_products = Table.from_pandas(
        data_products
        .to_pandas()
        .query('dataproduct_type == "image"')
        .query('obs_collection == "HST"')
        .query('type == "D"')
        .query('calib_level == 3')
        .query('productType == "SCIENCE"')
        .query('productSubGroupDescription == "DRZ"')
    )
    
    if len(download_products) < 0.5:
        continue

    obs_id_url = download_products['obs_id'][0]

    url = use_fitscut(obs_id_url,ra,dec,100,300)
    
    try:
        wget.download(url,out=f'C:/Users/oryan/Documents/esac-project/cutouts/hubble-training-100-cgi/{zooniverse_id}_cgi.png')
        downloaded = True
    except:
        time.sleep(0.1)
        continue
    time.sleep(0.1)

KeyboardInterrupt: 

In [ ]:
url